In [1]:
import numpy as np
import logging
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from nergrit_non_contextual_embeddings import NergritNonContextualEmbeddings


app = NergritNonContextualEmbeddings()

In [2]:
app.init_data(ignore_case=True)

using cached result from '.cache/nergrit:train_df.pkl'
using cached result from '.cache/nergrit:validation_df.pkl'
using cached result from '.cache/nergrit:test_series.pkl'
using cached result from '.cache/nergrit:test_sentence_series.pkl'
using cached result from '.cache/nergrit:uncased_vocab_series.pkl'


In [3]:
w2vmodel = app.build_word2vec_model()
words = app.get_words()

using cached result from '.cache/nergrit-non-contextual-embeddings:word2vec_model.pkl'
using cached result from '.cache/nergrit-non-contextual-embeddings:vocab_words.pkl'


In [4]:
from tensorflow.keras.utils import to_categorical

X_train, y_train = app.build_Xy_train(w2vmodel)
X_validation, y_validation = app.build_Xy_validation(w2vmodel)

def reshape(X):
    m, n = np.shape(X)
    return np.reshape(X, (m,n,1))

le = preprocessing.LabelEncoder()
le.fit(y_train)

def convert_categorical(Y):
    Y = le.transform(Y)
    return to_categorical(Y, len(set(Y)))

X_train = reshape(X_train)
y_train = convert_categorical(y_train)

X_validation = reshape(X_validation)
y_validation = convert_categorical(y_validation)

print(np.shape(X_train), np.shape(y_train))
print(np.shape(X_validation), np.shape(y_validation))

using cached result from '.cache/nergrit-non-contextual-embeddings:word2vec_train.pkl'
using cached result from '.cache/nergrit-non-contextual-embeddings:word2vec_validation.pkl'
(56210, 100, 1) (56210, 7)
(6983, 100, 1) (6983, 7)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, SimpleRNN, TimeDistributed, Dense
from tensorflow.keras import models

model = Sequential()
model.add(Embedding(len(X_train), 100, input_length=1))
model.add(LSTM(32, activation='tanh', input_shape = X_train.shape[1:]))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# model = models.load_model("lstm.model")

2022-03-27 10:47:48.121579: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-27 10:47:48.123358: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 100)            5621000   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                17024     
_________________________________________________________________
dense (Dense)                (None, 7)                 231       
Total params: 5,638,255
Trainable params: 5,638,255
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=3,
    validation_data=(X_validation, y_validation),
)

Epoch 1/3


2022-03-27 10:47:48.802408: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1757/1757 [==============================] - 372s 211ms/step - loss: 0.7964 - accuracy: 0.8190 - val_loss: 0.7899 - val_accuracy: 0.8193
Epoch 2/3
1757/1757 [==============================] - 354s 202ms/step - loss: 0.7844 - accuracy: 0.8200 - val_loss: 0.7863 - val_accuracy: 0.8193
Epoch 3/3
1757/1757 [==============================] - 370s 211ms/step - loss: 0.7843 - accuracy: 0.8200 - val_loss: 0.7861 - val_accuracy: 0.8193


In [7]:
model.save("lstm2.model")

2022-03-27 11:06:07.420382: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: lstm2.model/assets


INFO:tensorflow:Assets written to: lstm2.model/assets


In [8]:
y_pred = model.predict_classes(X_validation)

/home/nafkhanzam/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


y_pred = rnn_model.predict(X_validation)

In [9]:
pred_labels = le.inverse_transform(y_pred)

In [16]:

X_validation, y_validation = app.build_Xy_validation(w2vmodel)
np.shape(y_validation)
y_validation

using cached result from '.cache/nergrit-non-contextual-embeddings:word2vec_validation.pkl'


array([6, 6, 6, ..., 6, 6, 6])

In [11]:
accuracy = accuracy_score(y_validation, y_pred)
precision = precision_score(y_validation, y_pred, average='macro')
recall = recall_score(y_validation, y_pred, average='macro')
f1 = f1_score(y_validation, y_pred, average='macro')

print("Accuracy = {0:.2f}".format(accuracy))
print("Precision = {0:.2f}".format(precision))
print("Recall = {0:.2f}".format(recall))
print("F1 = {0:.2f}".format(f1))

Accuracy = 0.82
Precision = 0.12
Recall = 0.14
F1 = 0.13


/home/nafkhanzam/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
y_test_preds = []

for X_test in app.build_X_sentences_test_iter(w2vmodel):
    X_test = reshape(X_test)
    y_test = model.predict_classes(X_test)
    test_labels = le.inverse_transform(y_test)
    y_test_preds.append(test_labels)

/home/nafkhanzam/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [13]:
import pandas as pd
import numpy as np
np.set_string_function(lambda x: repr(list(x)), repr=False)
np.set_printoptions(linewidth=np.inf)

result_df = pd.DataFrame({'label': y_test_preds}).reset_index()
result_df.to_csv('pred.txt', index=False)

In [14]:
model.predict_classes(reshape([w2vmodel.wv['karangasem']]))

array([6])